In [20]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

# Local imports
from stability_testing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
pv_dest_path = './data/hydro/existing_plants/complementarity/pv_time_series'
wind_dest_path = './data/hydro/existing_plants/complementarity/wind_time_series'
pv_filename_prefix = 'cf_pv_profile'
wind_filename_prefix = 'cf_wind_profile'

c_stab_path = './data/hydro/existing_plants/complementarity/c_stab'
pearsons_path = './data/hydro s/existing_plants/complementarity/pearson'

rep_year_df = pd.read_csv('./data/hydro/misc/ed_representative_year.csv')
years = range(2007,2014)
months = range(1,13)
pv_wind_id_df = pd.read_csv('./data/hydro/misc/master_pv_wind_ed.csv')

# build the base dataframe with the timestamps from 2012
rng = pd.date_range(f'2012-01-01 00:00:00+00:00', periods=8760, freq='1H')
base_df = pd.DataFrame()
base_df['dateTime'] = rng 

In [22]:
pv_wind_id_df.drop_duplicates(subset=['site_id'], inplace=True)
pv_wind_id_df.reset_index(inplace=True, drop=True)
pv_wind_id_df

,lat,lon,dam_name,site_id,post_csv_filename,pv_id,wind_id
0,45.158051,-67.401657,Lower Saint Croix River,1021000,./data/hydro/existing_plants/synthetic_profile...,1681,1681
1,44.786598,-71.124901,Stearns Brook-Androscoggin River,1053500,./data/hydro/existing_plants/synthetic_profile...,1178,1178
2,44.456947,-71.186066,Stearns Brook-Androscoggin River,1054000,./data/hydro/existing_plants/synthetic_profile...,726,726
3,44.542599,-70.546501,Swift River-Androscoggin River,1054500,./data/hydro/existing_plants/synthetic_profile...,780,780
4,43.534500,-71.468102,Winnipesaukee River,1081000,./data/hydro/existing_plants/synthetic_profile...,1864,1864
...,...,...,...,...,...,...,...
123,40.596603,-111.423592,Lake Creek-Provo River,10155200,./data/hydro/existing_plants/synthetic_profile...,149,149
124,38.285801,-112.443298,South Creek-Beaver River,10234500,./data/hydro/existing_plants/synthetic_profile...,416,416
125,39.462898,-119.065804,Soda Lake-Carson River,10312275,./data/hydro/existing_plants/synthetic_profile...,69,69
126,39.470447,-120.104553,Little Truckee River,10344500,./data/hydro/existing_plants/synthetic_profile...,55,55


# 1. Create Montly Stability Coefficients

## Complementarity PV-Hydro

In [23]:
from pathlib import Path
from scipy.stats import pearsonr
import seaborn as sns

### Stability coefficient
missing_sites = []
df_missing = pd.DataFrame()
pv_wind_id_df.fillna(0, inplace=True)
pearsons_pv_hy = pd.DataFrame(columns=['site_id','p'])
annual_mean_p = pd.DataFrame()
annual_stab_df = pd.DataFrame(columns = ['site_id', 'lat', 'lon'])

for idx, row in pv_wind_id_df.iterrows():
    
    if row['post_csv_filename'] == 0:
        print( f"No file for site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue
    
    if row['site_id'] not in rep_year_df.site.values:
        print(f"Site doesn't have enough data: {row['site_id']}")
        empty_data_sites.append(row['site_id'])
        query_df.loc[idx,'post_csv_filename'] = ""
        continue
        
    print(f"{idx}. Processing PV profile {row['pv_id']} and NPD {row['site_id']}, {row['post_csv_filename']}")

    solar_df = pd.read_csv(f"{pv_dest_path}/{row['pv_id']}_{pv_filename_prefix}.csv",parse_dates=True,index_col=0)

    hydro_df = pd.read_csv(row['post_csv_filename'],parse_dates=True,index_col=0)
    hydro_df.fillna(0, inplace=True)
    
    if float(hydro_df['capacity_factor'].sum()) == 0.0:
        print( f"No data in site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue

    c_stab = []
    pearsons = []
    
    # Finds the representative year for the given stream gage
    year = str(rep_year_df['year'].loc[rep_year_df.site == row['site_id']].values[0])
    
    # Massaging the timestamps of the hydropower time-series given that we are chosing the best year in terms of data completion.
    # Here, we copy the CF into a new dataframe with the time-stamps from 2012
    tmp_hydro_df_rep_year = hydro_df.loc[year,'capacity_factor'].copy().to_frame()
    tmp_hydro_df = base_df.copy()
    tmp_hydro_df.set_index('dateTime',inplace=True)
    tmp_hydro_df['capacity_factor'] = tmp_hydro_df_rep_year.loc[year,'capacity_factor'].values
     
    tmp_solar_df = solar_df.loc['2012-01-01':'2012-12-30'].copy()

    for month in months:

        if month < 10:
            month = f'0{str(month)}'
        else:
            month = str(month)
        
        tmp_hydro_monthly_df = tmp_hydro_df.loc[f'2012-{month}','capacity_factor'].copy().to_frame()
        tmp_solar_monthly_df = tmp_solar_df.loc[f'2012-{month}'].copy()

        tmp_hydro_monthly_df.rename(columns={'capacity_factor':row['site_id']}, inplace=True)
        tmp_solar_monthly_df.rename(columns={str(row['pv_id']):row['site_id']}, inplace=True)

        # Compute complementarity metrics (PV as reference)
        _, c_stab_tmp = main_stability(tmp_solar_monthly_df, tmp_hydro_monthly_df)
        c_stab_tmp.rename(columns={row['site_id']:month}, inplace=True)

        c_stab.append(c_stab_tmp)
         
        corr, _ = pearsonr(tmp_solar_monthly_df[row['site_id']].values, tmp_hydro_monthly_df[row['site_id']].values)
        pearsons.append(corr)
    
    # ***************************************** Stability Coefficient *****************************************
    c_stab_df = pd.concat(c_stab, axis=1)

    annual_mean = c_stab_df.mean(axis=0)

    annual_stab_df.loc[idx,'site_id'] = row['site_id']
    annual_stab_df.loc[idx,'pv_id'] = row['pv_id']
    annual_stab_df.loc[idx,'lat'] = row['lat']
    annual_stab_df.loc[idx,'lon'] = row['lon']
    annual_stab_df.loc[idx,months] = annual_mean.values
    
    # ***************************************** Pearsons Coefficient *****************************************
    p_series = pd.Series(pearsons)

    annual_mean_p.loc[idx,'site_id'] = row['site_id']
    annual_mean_p.loc[idx,'pv_id'] = row['pv_id']
    annual_mean_p.loc[idx,'lat'] = row['lat']
    annual_mean_p.loc[idx,'lon'] = row['lon']
    annual_mean_p.loc[idx,months] = p_series.values

annual_stab_df.to_csv(f"{c_stab_path}/rep_year_MONTHLY_ed_stab_pv_hydro.csv")
annual_mean_p.to_csv(f"{pearsons_path}/rep_year_MONTHLY_ed_pearsons_pv_hydro.csv")

0. Processing PV profile 1681 and NPD 1021000, ./data/hydro/existing_plants/synthetic_profiles/ed_1021000_2010_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


1. Processing PV profile 1178 and NPD 1053500, ./data/hydro/existing_plants/synthetic_profiles/ed_1053500_2013_synthetic.csv
2. Processing PV profile 726 and NPD 1054000, ./data/hydro/existing_plants/synthetic_profiles/ed_1054000_2013_synthetic.csv
3. Processing PV profile 780 and NPD 1054500, ./data/hydro/existing_plants/synthetic_profiles/ed_1054500_2014_synthetic.csv
No data in site 1054500 
4. Processing PV profile 1864 and NPD 1081000, ./data/hydro/existing_plants/synthetic_profiles/ed_1081000_2014_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


5. Processing PV profile 1772 and NPD 1082000, ./data/hydro/existing_plants/synthetic_profiles/ed_1082000_2017_synthetic.csv
No data in site 1082000 
6. Processing PV profile 1180 and NPD 1091500, ./data/hydro/existing_plants/synthetic_profiles/ed_1091500_2019_synthetic.csv


...

/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


122. Processing PV profile 108 and NPD 10140700, ./data/hydro/existing_plants/synthetic_profiles/ed_10140700_2012_synthetic.csv
No data in site 10140700 
123. Processing PV profile 149 and NPD 10155200, ./data/hydro/existing_plants/synthetic_profiles/ed_10155200_2010_synthetic.csv
124. Processing PV profile 416 and NPD 10234500, ./data/hydro/existing_plants/synthetic_profiles/ed_10234500_2015_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


125. Processing PV profile 69 and NPD 10312275, ./data/hydro/existing_plants/synthetic_profiles/ed_10312275_2012_synthetic.csv
126. Processing PV profile 55 and NPD 10344500, ./data/hydro/existing_plants/synthetic_profiles/ed_10344500_2013_synthetic.csv
127. Processing PV profile 270 and NPD 11465200, ./data/hydro/existing_plants/synthetic_profiles/ed_11465200_2012_synthetic.csv
No data in site 11465200 


FileNotFoundError: [Errno 2] No such file or directory: './data/hydro s/existing_plants/complementarity/pearson/rep_year_MONTHLY_ed_pearsons_pv_hydro.csv'

In [ ]:
annual_stab_df.head()

## Complementarity PV-Wind

In [ ]:
from pathlib import Path
from scipy.stats import pearsonr

### Stability coefficient
missing_sites = []
df_missing = pd.DataFrame()

pv_wind_id_df.fillna(0, inplace=True)

annual_mean_p = pd.DataFrame()
pearsons_wind_hy = pd.DataFrame(columns=['site_id','p'])

annual_stab_df = pd.DataFrame(columns = ['site_id', 'lat', 'lon'])

for idx, row in pv_wind_id_df.iterrows():
    
    if row['post_csv_filename'] == 0:
        print( f"No file for site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue
    
    if row['site_id'] not in rep_year_df.site.values:
        print(f"Site doesn't have enough data: {row['site_id']}")
        empty_data_sites.append(row['site_id'])
        query_df.loc[idx,'post_csv_filename'] = ""
        continue

    print(f"{idx}. Processing Wind profile {row['wind_id']} and NPD {row['site_id']}, {row['post_csv_filename']}")

    wind_df = pd.read_csv(f"{wind_dest_path}/{row['wind_id']}_{wind_filename_prefix}.csv",parse_dates=True,index_col=0)

    hydro_df = pd.read_csv(row['post_csv_filename'],parse_dates=True,index_col=0)
    hydro_df.fillna(0, inplace=True)

    if float(hydro_df['capacity_factor'].sum()) == 0.0:
        print( f"No data in site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue

    cf_mix = []
    c_stab = []
    pearsons = []
    
    # Finds the representative year for the given stream gage
    year = str(rep_year_df['year'].loc[rep_year_df.site == row['site_id']].values[0])
    
    # Massaging the timestamps of the hydropower time-series given that we are chosing the best year in terms of data completion.
    # Here, we copy the CF into a new dataframe with the time-stamps from 2012
    tmp_hydro_df_rep_year = hydro_df.loc[year,'capacity_factor'].copy().to_frame()
    tmp_hydro_df = base_df.copy()
    tmp_hydro_df.set_index('dateTime',inplace=True)
    tmp_hydro_df['capacity_factor'] = tmp_hydro_df_rep_year.loc[year,'capacity_factor'].values
    
    tmp_wind_df = wind_df.loc['2012-01-01':'2012-12-30'].copy()

    
    for month in months:

        if month < 10:
            month = f'0{str(month)}'
        else:
            month = str(month)
        
        tmp_hydro_monthly_df = tmp_hydro_df.loc[f'2012-{month}','capacity_factor'].copy().to_frame()
        tmp_wind_monthly_df = tmp_wind_df.loc[f'2012-{month}'].copy()

        tmp_hydro_monthly_df.rename(columns={'capacity_factor':row['site_id']}, inplace=True)
        tmp_wind_monthly_df.rename(columns={str(row['wind_id']):row['site_id']}, inplace=True)

        # Compute complementarity metrics (PV as reference)
        _, c_stab_tmp = main_stability(tmp_wind_monthly_df, tmp_hydro_monthly_df)
        c_stab_tmp.rename(columns={row['site_id']:month}, inplace=True)

        c_stab.append(c_stab_tmp)
        
        corr, _ = pearsonr(tmp_wind_monthly_df[row['site_id']].values, tmp_hydro_monthly_df[row['site_id']].values)
        pearsons.append(corr)

    c_stab_df = pd.concat(c_stab, axis=1)
    
    annual_mean = c_stab_df.mean(axis=0)

    annual_stab_df.loc[idx,'site_id'] = row['site_id']
    annual_stab_df.loc[idx,'pv_id'] = row['pv_id']
    annual_stab_df.loc[idx,'lat'] = row['lat']
    annual_stab_df.loc[idx,'lon'] = row['lon']
    annual_stab_df.loc[idx,months] = annual_mean.values
    
    # ***************************************** Pearsons Coefficient *****************************************
    p_series = pd.Series(pearsons)

    annual_mean_p.loc[idx,'site_id'] = row['site_id']
    annual_mean_p.loc[idx,'pv_id'] = row['pv_id']
    annual_mean_p.loc[idx,'lat'] = row['lat']
    annual_mean_p.loc[idx,'lon'] = row['lon']
    annual_mean_p.loc[idx,months] = p_series.values
    #break

annual_stab_df.to_csv(f"{c_stab_path}/rep_year_MONTHLY_ed_stab_wind_hydro.csv")
annual_mean_p.to_csv(f"{pearsons_path}/rep_year_MONTHLY_ed_pearsons_wind_hydro.csv")

# Relationship plots

## Ranking the sites based on their annual stability coefficient

### ED PV-Hydro

In [ ]:
pv_hydro_annual_stab_df = pd.read_csv(f"{c_stab_path}/rep_year_MONTHLY_ed_stab_pv_hydro.csv")
pv_hydro_annual_stab_df

# Cleaning up the dataframe
c_stab_pv_hydro_df = pd.DataFrame()
c_stab_pv_hydro_df['site_id'] = pv_hydro_annual_stab_df['site_id']
c_stab_pv_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']] = pv_hydro_annual_stab_df[['1','2','3','4','5','6','7','8','9','10','11','12']].copy()

# Computing the yearly mean across the months
rank_c_stab_pv_hydro_df = pd.DataFrame()
rank_c_stab_pv_hydro_df['site_id'] = c_stab_pv_hydro_df['site_id']
rank_c_stab_pv_hydro_df['c_stab_mean'] = c_stab_pv_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']].mean(axis=1)

In [ ]:
c_stab_pv_hydro_df

In [ ]:
rank_c_stab_pv_hydro_df.sort_values(by='c_stab_mean',ascending=True)

In [ ]:
# Calculating the deciles for the yearly mean
rank_c_stab_pv_hydro_df['decile_rank']= pd.qcut(rank_c_stab_pv_hydro_df['c_stab_mean'], 
                                         q = 10, labels = False)
rank_c_stab_pv_hydro_df.head()

In [ ]:
rank_c_stab_pv_hydro_df.to_csv('rank.csv')

In [ ]:
coo_df = pv_wind_id_df[['lat','lon','site_id','dam_name']]
input_df = pd.read_csv('../data/misc/flexpower_existing_hydro_w_NID_join.csv')
input_df_capacity = input_df[['stream_gauge_id','CH_MW']].copy()


coo_df=coo_df.merge(input_df_capacity, left_on='site_id', right_on='stream_gauge_id', how='outer')
coo_df.head()

In [ ]:
rank_c_stab_pv_hydro_df=rank_c_stab_pv_hydro_df.merge(coo_df, left_on='site_id', right_on='site_id', how='outer')

In [ ]:
rank_c_stab_pv_hydro_df.head()

In [ ]:
rank_c_stab_pv_hydro_df.to_csv('../data/existing_plants/complementarity/ed_pv_hydro_c_stab_decile_rank.csv')
rank_c_stab_pv_hydro_df.dropna(inplace=True)
rank_c_stab_pv_hydro_df

In [ ]:
how_many_on_each_decile = {}

for dec in rank_c_stab_pv_hydro_df.decile_rank.unique():
    how_many_on_each_decile[str(dec)] = rank_c_stab_pv_hydro_df.loc[rank_c_stab_pv_hydro_df.decile_rank == dec].count()


In [ ]:
import random

def pre_process_for_rank_plotting(rank_df, monthly_df):
    plotting_sites = []
    
    for dec in rank_df.decile_rank.unique():
        #print(dec)
        # Obtain the best site for each rank (mean)
        site = rank_df.loc[rank_df.decile_rank == dec].sort_values(by='c_stab_mean',ascending=False).site_id.values[0]
        print(f"Best site {site}, {rank_df.dam_name.loc[rank_df.site_id == site].values[0]} for decile {dec} with stability coefficient {rank_df.c_stab_mean.loc[rank_df.site_id == site].values}")
        tmp_tup = (site,dec)
        plotting_sites.append(tmp_tup)
       
    # Create an empty dataframe to re-organize the data for plotting line ranking
    no_of_Rows = len(pv_hydro_annual_stab_df)
    plotting_df = pd.DataFrame(index=range(no_of_Rows))
    plotting_df['site_id'] = 0
    plotting_df['Month'] = 0
    plotting_df['c_stab'] = 0
    plotting_df['rank'] = 0
    
    plot_df_list = []
    months = range(1,13)
    for month in months:
        #print(month)
        for idx, row in monthly_df.iterrows():
            #print(row)
            plotting_df.loc[idx,'site_id'] = row['site_id']
            plotting_df.loc[idx,'Month'] = month
            plotting_df.loc[idx,'c_stab'] = row[str(month)]
            plotting_df.loc[idx,'decile_rank'] = rank_df.decile_rank.loc[rank_df.site_id == row['site_id']].values[0]

        plot_df_list.append(plotting_df.copy())
    
    # This is the dataframe with all the sites and ranks, reformatted for line plot
    c_stab_plotting_data = pd.concat(plot_df_list)
    
    # This is the dataset that will contain only the top site for each rank. In this case, 10 sites
    #c_stab_min_rank_plot = pd.DataFrame(index=len(tmp_tup))
    tmp_list = []
    for item in plotting_sites:
        tmp_df = c_stab_plotting_data.loc[plotting_df.site_id == item[0]].copy()
        tmp_df['rank'] = item[1]
        tmp_list.append(tmp_df)
    c_stab_min_rank_plot = pd.concat(tmp_list)
    
    return c_stab_plotting_data, c_stab_min_rank_plot
        
        

In [ ]:
plot_df, min_plot_df = pre_process_for_rank_plotting(rank_c_stab_pv_hydro_df, c_stab_pv_hydro_df)

In [ ]:
min_plot_df

In [ ]:
min_plot_df = min_plot_df.astype({"site_id": str})
min_plot_df.reset_index(inplace=True)
min_plot_df.sort_values(by='rank', ascending=False,inplace=True)

In [ ]:
import seaborn as sns

sns.set()
rel = sns.relplot(
    data=min_plot_df,
    x="Month", y="c_stab",
    kind="line",hue="site_id",size="rank",
    height=7.5, aspect=1.5, facet_kws=dict(sharex=False),legend="full"
).set(title="The best NPDs for each decile in PV-Hydro complementarity")
#rel.fig.suptitle("The best NPDs for each decile")

1. take the average across the months for each site
2. rank the table based on the value of the average
3. bin this data to obtain "top" percentile (e.g. 20th)
4. assign a color to each bin (e.g. top 20th yellow,...). Use the same color scale that NREL uses
5. Use this in a map

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#sns.set()
sns.set_theme(style="dark")
#sns.set_context("talk")
# Plot each year's time series in its own facet
g = sns.relplot(
    data=min_plot_df,
    x="Month", y="c_stab", col="site_id", hue="rank",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=5, height=4, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for site, ax in g.axes_dict.items():
    
    rank = min_plot_df["rank"].loc[min_plot_df.site_id == site].values[0]
    # Add the title as an annotation within the plot
    ax.text(.1, .95, f"Existing plant: {site}, Rank: {rank}", transform=ax.transAxes, fontweight="bold")
    
    # Add the title as an annotation within the plot
    
    #ax.text(.8, .65, f"Rank: {rank}", transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=min_plot_df, x="Month", y="c_stab", units="site_id",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )
    ax.set_ylim([0,1])

# Reduce the frequency of the x axis ticks
#ax.set_xticks(ax.get_xticks()[::2])
# Tweak the supporting aspects of the plot
g.set_titles("")
#g.set_axis_labels("Months", "Stability Coefficient")
#g.fig.suptitle("The best NPDs for each decile")
g.tight_layout()
plt.savefig("ed_ranked_pv_hydro_deciles.eps")

### Existing Wind-Hydro

In [ ]:
wind_hydro_annual_stab_df = pd.read_csv(f"{c_stab_path}/rep_year_MONTHLY_ed_stab_wind_hydro.csv")
wind_hydro_annual_stab_df

# Cleaning up the dataframe
c_stab_wind_hydro_df = pd.DataFrame()
c_stab_wind_hydro_df['site_id'] = wind_hydro_annual_stab_df['site_id']
c_stab_wind_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']] = wind_hydro_annual_stab_df[['1','2','3','4','5','6','7','8','9','10','11','12']].copy()

# Computing the yearly mean across the months
rank_c_stab_wind_hydro_df = pd.DataFrame()
rank_c_stab_wind_hydro_df['site_id'] = c_stab_wind_hydro_df['site_id']
rank_c_stab_wind_hydro_df['c_stab_mean'] = c_stab_wind_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']].mean(axis=1)

In [ ]:
rank_c_stab_wind_hydro_df.sort_values(by='c_stab_mean',ascending=True)

# Calculating the deciles for the yearly mean
rank_c_stab_wind_hydro_df['decile_rank']= pd.qcut(rank_c_stab_wind_hydro_df['c_stab_mean'], 
                                         q = 10, labels = False)

coo_df = pv_wind_id_df[['lat','lon','site_id','dam_name']]
input_df = pd.read_csv('../data/misc/flexpower_existing_hydro_w_NID_join.csv')
input_df_capacity = input_df[['stream_gauge_id','CH_MW']].copy()


coo_df=coo_df.merge(input_df_capacity, left_on='site_id', right_on='stream_gauge_id', how='outer')
coo_df.head()



rank_c_stab_wind_hydro_df=rank_c_stab_wind_hydro_df.merge(coo_df, left_on='site_id', right_on='site_id', how='outer')

rank_c_stab_wind_hydro_df.to_csv('../data/existing_plants/complementarity/ed_wind_hydro_c_stab_decile_rank.csv')
rank_c_stab_wind_hydro_df.dropna(inplace=True)
rank_c_stab_wind_hydro_df


In [ ]:
plot_wind_hydro_df, min_wind_hydro_plot_df = pre_process_for_rank_plotting(rank_c_stab_wind_hydro_df, c_stab_wind_hydro_df)
min_plot_df = min_wind_hydro_plot_df.astype({"site_id": str})
min_plot_df.reset_index(inplace=True)

In [ ]:
min_wind_hydro_plot_df

In [ ]:
min_wind_hydro_plot_df = min_wind_hydro_plot_df.astype({"site_id": str})
min_wind_hydro_plot_df.reset_index(inplace=True, drop=True)
min_wind_hydro_plot_df.sort_values(by='rank', ascending=False,inplace=True)

In [ ]:
#coo_df = pv_wind_id_df[['lat','lon','site_id']]
#input_df = pd.read_csv('combined_hydro_stream_gauge_full2.csv')
#input_df_capacity = input_df[['stream_gauge_id','potential_cap_mw']].copy()

#coo_df=coo_df.merge(input_df_capacity, left_on='site_id', right_on='stream_gauge_id', how='outer')
#coo_df.head()

In [ ]:
#rank_c_stab_wind_hydro_df=rank_c_stab_wind_hydro_df.merge(coo_df, left_on='site_id', right_on='site_id', how='outer')
#rank_c_stab_wind_hydro_df.head()

In [ ]:
rank_c_stab_wind_hydro_df.to_csv('../data/npd/complementarity/ed_wind_hydro_c_stab_decile_rank.csv')

In [ ]:
import seaborn as sns

#sns.set()
sns.set_theme(style="dark")
#sns.set_context("talk")
# Plot each year's time series in its own facet
g = sns.relplot(
    data=min_wind_hydro_plot_df,
    x="Month", y="c_stab", col="site_id", hue="rank",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=5, height=4, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for site, ax in g.axes_dict.items():
    
    # Add the title as an annotation within the plot
    ax.text(.6, .75, f"NPD site: {site}", transform=ax.transAxes, fontweight="bold")
    
    # Add the title as an annotation within the plot
    rank = min_plot_df["rank"].loc[min_plot_df.site_id == site].values[0]
    ax.text(.8, .65, f"Rank: {rank}", transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=min_plot_df, x="Month", y="c_stab", units="site_id",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )
    ax.set_ylim([0,1])

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("Months", "Stability Coefficient")
#g.fig.suptitle("The best NPDs for each decile")
g.tight_layout()
plt.savefig("ed_ranked_wind_hydro_deciles.eps")